In [1]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth
from sklearn.cluster import KMeans
import pickle
import numpy as np
from IPython.utils import capture

In [2]:
# import chart dataframe
chart=pd.read_csv('chart')
chart_lowercase=pd.read_csv('chart_lowercase')
df_keggle=pd.read_csv('df_keggle')

In [4]:
# connection to Spotify
secrets_file = open("C:/Ironhack/Week6/6.5 API wrappers, Spotify API/6.5 API wrappers, Spotify API/secrets.txt", "r")
string = secrets_file.read()
secrets_dict = {}

with capture.capture_output():
    for line in string.split('\n'):
        if len(line) > 0:
            print(line.split(':'))
            secrets_dict[line.split(':')[0]] = line.split(':')[1].strip()

# Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                           client_secret=secrets_dict['clientsecret']))

In [5]:
import pickle
import spotipy
import pandas as pd
import numpy as np
from spotipy.oauth2 import SpotifyClientCredentials
import random

# Function to load scaler
def load_scaler(scaler_path):
    return pickle.load(open(scaler_path, 'rb'))

# Function to load kmeans model
def load_kmeans_model(model_path):
    return pickle.load(open(model_path, 'rb'))

# Function to recommend a song from a cluster
def recommend_song_from_cluster(cluster_number, cluster_data):
    same_cluster_songs = cluster_data[cluster_data['cluster'] == cluster_number]
    recommended_song = same_cluster_songs.sample(1)['track_name'].values[0]
    return recommended_song

# Function to get the next search result from Spotify
def get_next_search_result(sp, input_track):
    # Perform the next search
    next_search_result = sp.search(q=input_track, limit=2)
    
    # Return the second item in the search results
    return next_search_result['tracks']['items'][1] if len(next_search_result['tracks']['items']) > 1 else None

# Load scaler and kmeans model
scaler_playlist = load_scaler('scaler.pk')
kmeans = load_kmeans_model('kmeans.pk')

# Prompt the user to input a song and artist
input_track = input("Enter the song name: ")
input_artist = input("Enter the artist name: ")

# Check if the input song and artist are in the Hot 100
if chart['track_name'].str.contains(input_track, case=False).any() and chart['artist'].str.contains(input_artist, case=False).any():
    # Recommend another song from the Hot 100
    hot100_recommendation = chart[(~chart['track_name'].str.contains(input_track, case=False)) & (~chart['artist'].str.contains(input_artist, case=False))].sample(1).iloc[0]
    song_name = hot100_recommendation['track_name']
    artist_name = hot100_recommendation['artist']
    print(f"Your recommended song is: {song_name} by {artist_name}")
else:
    # Initialize SpotiPy with user credentials
    secrets_file = open("C:/Ironhack/Week6/6.5 API wrappers, Spotify API/6.5 API wrappers, Spotify API/secrets.txt","r")
    string = secrets_file.read()
    secrets_dict={}
    with capture.capture_output():
        for line in string.split('\n'):
            if len(line) > 0:
                print(line.split(':'))
                secrets_dict[line.split(':')[0]] = line.split(':')[1].strip()

    # Initialize SpotiPy with user credentials
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                            client_secret=secrets_dict['clientsecret']))
    get_input_song_id = sp.search(q=f"{input_track} artist:{input_artist}", limit=50)
    input_song_as_df = pd.json_normalize(get_input_song_id["tracks"]["items"])
    track_info = get_input_song_id['tracks']['items'][0]

    # Song ID
    song_id = track_info['id']

    # Song name
    song_name = track_info['name']

    # Artist name
    artist_name = track_info['artists'][0]['name']

    # Album information
    album_info = track_info['album']

    # Album name
    album_name = album_info['name']

    # Print the information
    print(f'Is your input song: {song_name} by {artist_name}')

    # Ask the user to confirm the choice
    user_confirmation = input("Is this the correct song? (yes/no): ").lower()

    # If the user confirms, proceed with the recommendation
    if user_confirmation == 'yes':
        # Get the audio features for a song
        input_song_audio_features = sp.audio_features(song_id)
        input_song_audio_features = pd.DataFrame(input_song_audio_features)

        # Scale the audio features for the input song
        numeric_columns = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
                        'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
                        'duration_ms']

        # Check if 'cluster' column exists before using it
        if 'cluster' in input_song_audio_features.columns:
            # Exclude 'cluster' column from the list of numeric columns
            numeric_columns = [col for col in numeric_columns if col != 'cluster']

        scaled_input_audio_features = scaler_playlist.transform(input_song_audio_features[numeric_columns])

        # Predict the cluster for the input song 
        input_song_cluster = kmeans.predict(scaled_input_audio_features)  # Removed [0]

        # Extract the cluster label
        cluster_label = list(input_song_cluster)[0]

        # Filter the dataframe to get songs in the same cluster
        df_cluster = df_keggle[df_keggle['cluster'] == cluster_label]

        # Randomly choose a recommendation from the same cluster
        recommendation_row = df_cluster.sample(1).iloc[0]

        # Extract the song and artist from the recommendation
        recommended_song = recommendation_row['track_name']
        recommended_artist = recommendation_row['artist']

        print(f'Your recommended song is: {recommended_song} by {recommended_artist}')

        

    # If the user does not confirm, choose the second song from the Spotify list
    else:
        # Assuming you have retrieved information about the next search result using Spotify API
        next_search_result = get_next_search_result(sp, input_track)  # Provide sp and input_track as arguments
        track_info = next_search_result['tracks']['items'][1]  # Choose the second song


Is your input song: Baby by Justin Bieber
Your recommended song is: Bad Boyz by Shyne Featuring Barrington Levy


c:\Users\usuario\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
